<a href="https://colab.research.google.com/github/Sh-Hi-Go/DTDL_findings/blob/main/VECTOR_SIMILARITY_WITH_FAISS_SUCCESSFUL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SUCCESSFUL ATTEMPT ON COMPARING FAISS INDEX
IVF came out to be the best in terms of response time. All the indices gave same outputs though as the dataset was small.

In [3]:
!pip install sentence_transformers
!pip install faiss-gpu
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 44.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import time
from sentence_transformers import SentenceTransformer
import faiss
import torch

In [27]:
df = pd.read_csv("/content/vector_similarity_search_dataset.csv")
print(len(df))
df.head()

45


,question,combined_response
0,How can I implement user authentication in a M...,Consider using industry-standard libraries lik...
1,How to best structure a large React applicatio...,"Utilize modular design principles, break down ..."
2,What are some effective approaches to optimize...,Explore state management libraries like Redux ...
3,How can I create accessible and inclusive user...,"Follow WCAG guidelines, use semantic HTML elem..."
4,What are the key considerations for deploying ...,"Choose a reliable hosting platform, implement ..."


In [28]:
print(df.index)

RangeIndex(start=0, stop=45, step=1)


In [6]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# pre-process data
questions_base = df["question"].tolist()
# embed the data
questions_base_embds=model.encode(questions_base)

In [9]:
dimensions = questions_base_embds.shape[1]

### Flat Index

In [12]:
flat_index = faiss.IndexFlatIP(dimensions)
flat_index.add(questions_base_embds)
faiss.write_index(flat_index, 'flat_questions_base')
flat_index = faiss.read_index('flat_questions_base')

### Locality Sensitive Hashing

In [13]:
n_bits = dimensions*4
LSH_index = faiss.IndexLSH(dimensions,n_bits)
LSH_index.add(questions_base_embds)
faiss.write_index(LSH_index, 'LSH_questions_base')
LSH_index = faiss.read_index('LSH_questions_base')

#### HNSW

In [14]:
M = 8 #number of connections each vertex has
ef_search = 4 #depth of search, high value slow but accurate
ef_construction = 16 #how much the N/W will search during construction of N/W

In [15]:
hnsw_index = faiss.IndexHNSWFlat(dimensions, M)
hnsw_index.hnsw.efSearch = ef_search
hnsw_index.hnsw.efConstruction = ef_construction

In [16]:
hnsw_index.add(questions_base_embds)
faiss.write_index(hnsw_index, 'hnsw_questions_base')
hnsw_index = faiss.read_index('hnsw_questions_base')

### Inverted File Index(IVF)

In [17]:
n_list = 4 #number of centroids in our vector store
quantiser = faiss.IndexFlatIP(dimensions) #defines how we do the search among the selected vectors

In [18]:
IVF_index = faiss.IndexIVFFlat(quantiser, dimensions, n_list)

In [19]:
IVF_index.train(questions_base_embds) #it is the only index which is not trained before

In [20]:
IVF_index.add(questions_base_embds)
IVF_index.nprobe=1
faiss.write_index(IVF_index, 'IVF_questions_base')
IVF_index = faiss.read_index('IVF_questions_base')

### EVALUATING THE DIFFERENT INDICES

In [21]:
query = "For a particular task how to measure LLM performance"

In [33]:
%%time
query_vector = model.encode([query])
k = 2
top_k = flat_index.search(query_vector, k)  # top2 only

for id in top_k[1].tolist()[0]:
  print("index: ", id)
  print("question: ", (df.iloc[id])["question"])
  print("answer: ", (df.iloc[id])["combined_response"])
  print("\n")

index:  18
question:  How can I evaluate the performance of an LLM on a specific task?
answer:  Use metrics like perplexity, BLEU score, or task-specific accuracy based on your objectives and dataset.


index:  24
question:  How can I compare the performance of different LLMs on a task?
answer:  Evaluate using benchmark datasets and metrics, considering factors like model size, training data, and task specificity.


CPU times: user 16.5 ms, sys: 14 µs, total: 16.5 ms
Wall time: 16.2 ms


In [34]:
%%time
query_vector = model.encode([query])
k = 2
top_k = LSH_index.search(query_vector, k)  # top2 only

for id in top_k[1].tolist()[0]:
  print("index: ", id)
  print("question: ", (df.iloc[id])["question"])
  print("answer: ", (df.iloc[id])["combined_response"])
  print("\n")

index:  18
question:  How can I evaluate the performance of an LLM on a specific task?
answer:  Use metrics like perplexity, BLEU score, or task-specific accuracy based on your objectives and dataset.


index:  24
question:  How can I compare the performance of different LLMs on a task?
answer:  Evaluate using benchmark datasets and metrics, considering factors like model size, training data, and task specificity.


CPU times: user 13.6 ms, sys: 0 ns, total: 13.6 ms
Wall time: 13.4 ms


In [35]:
%%time
query_vector = model.encode([query])
k = 2
top_k = hnsw_index.search(query_vector, k)  # top2 only

for id in top_k[1].tolist()[0]:
  print("index: ", id)
  print("question: ", (df.iloc[id])["question"])
  print("answer: ", (df.iloc[id])["combined_response"])
  print("\n")

index:  18
question:  How can I evaluate the performance of an LLM on a specific task?
answer:  Use metrics like perplexity, BLEU score, or task-specific accuracy based on your objectives and dataset.


index:  24
question:  How can I compare the performance of different LLMs on a task?
answer:  Evaluate using benchmark datasets and metrics, considering factors like model size, training data, and task specificity.


CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 17 ms


In [36]:
%%time
query_vector = model.encode([query])
k = 2
top_k = IVF_index.search(query_vector, k)  # top2 only

for id in top_k[1].tolist()[0]:
  print("index: ", id)
  print("question: ", (df.iloc[id])["question"])
  print("answer: ", (df.iloc[id])["combined_response"])
  print("\n")

index:  18
question:  How can I evaluate the performance of an LLM on a specific task?
answer:  Use metrics like perplexity, BLEU score, or task-specific accuracy based on your objectives and dataset.


index:  24
question:  How can I compare the performance of different LLMs on a task?
answer:  Evaluate using benchmark datasets and metrics, considering factors like model size, training data, and task specificity.


CPU times: user 12.1 ms, sys: 4.02 ms, total: 16.1 ms
Wall time: 17.3 ms
